In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import poplib
import email
import yaml
import pickle
with open("/content/drive/MyDrive/credentials.yml") as f:
    my_credentials = yaml.load(f, Loader=yaml.FullLoader)
user, password = my_credentials["user"], my_credentials["password"]
pop3_server = 'pop.gmail.com'
my_mail = poplib.POP3_SSL(pop3_server)
my_mail.user(user)
my_mail.pass_(password)
num_messages = len(my_mail.list()[1])
msgs = []
for i in range(num_messages, 0, -1):
    response, msg_lines, octets = my_mail.retr(i)
    msg_data = b'\r\n'.join(msg_lines).decode('utf-8')
    msgs.append(msg_data)
my_mail.quit()
list1 = []
for msg in msgs:
    my_msg = email.message_from_string(msg)
    print("_______________")
    print("subj:", my_msg['subject'])
    print("from:", my_msg['from'])
    print("body:")
    email_content = ""
    for part in my_msg.walk():
        if part.get_content_type() == 'text/plain':
            email_content = part.get_payload()
            print(email_content)
            list1.append(email_content)
print(len(list1))
with open('/content/drive/MyDrive/svm_classifier.pkl', 'rb') as f:
    classifier, cv, _, _ = pickle.load(f)
X_test_cv = cv.transform(list1)
predicted_categories = classifier.predict(X_test_cv)
print("Predicted Categories:")
for i, category in enumerate(predicted_categories):
    print(f"Email {i + 1}: {category}")

_______________
subj: 
from: U RAKSHITHA <21211a7259@bvrit.ac.in>
body:
AJIOMania LIVE NOW | 50-90% off* on 5000+ brands

-- 
*Engineering **Sciences*

*BVRIT <http://bvrit.ac.in>* | *SVECW* 
<http://svecw.edu.in> | *VIT* <http://vishnu.edu.in> | *BVRITH * 
<http://bvrithyderabad.edu.in>

*Medical Sciences*

*VDC 
<http://vdc.edu.in>* | *SVCP <http://svcp.edu.in>* | *VIPER  
<http://viper.ac.in>*| *BVRICE* <http://bvrice.edu.in>*  
<http://viper.ac.in>*

_______________
subj: 
from: Inti Kumaraswamy <kumaraswamy.inti@gmail.com>
body:
Respected student,
You are cordially invited to our conference on 15-10-2023

2
Predicted Categories:
Email 1: spam
Email 2: important


In [ ]:
predicted_categories

array(['spam', 'important'], dtype=object)

In [ ]:
list2=[]
for i in range(len(predicted_categories)):
  if predicted_categories[i]=="important":
    list2.append(list1[i])
len(list2)

In [ ]:
pip install sumy

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.utils import get_stop_words
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def summary(text):
  parser = PlaintextParser.from_string(text, Tokenizer("english"))
  summarizer = LsaSummarizer()
  summarizer.stop_words = get_stop_words("english")
  summary = summarizer(parser.document, sentences_count=1)
  summary_text = str(summary[0]).split()
  if len(summary_text) > 6:
        summary_text = ' '.join(summary_text[:6])
  return summary_text

In [ ]:
import spacy
import re
from dateutil.parser import parse
from datetime import timedelta
nlp = spacy.load("en_core_web_sm")
def preprocess_date_string(date_string):
    # Converts "21st", "22nd", etc. to "21", "22", etc.
    date_string = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_string)
    return date_string
def format_date_for_output(date):
    if date:
        return 'new Date("' + date.strftime('%a %b %d %Y %H:%M:%S %Z%z') + '")'
    else:
        return 'null'
def extract_event_name_from_subject(email_subject):
    doc = nlp(email_subject)
    event_name = ""
    event_keywords = ["conference", "workshop", "webinar", "lecture", "seminar","exam","course"]
    for token in doc:
        if token.text.lower() in event_keywords:
            event_name_start = doc[token.i:].text
            event_name = event_name_start.strip()
            break
    return event_name
def extract_event_info(email_body):
    doc = nlp(email_body)
    event_id = 122
    event_title = ""
    event_date = None
    event_description = ""
    admin_ids = [1, 2, 3, 4]
    event_keywords = ["event", "schedule", "meeting","event", "schedule", "meeting", "session", "conference", "workshop", "webinar", "presentation", "lecture", "seminar", "talk", "demo", "training", "class", "course", "forum",  "symposium", "gathering", "ceremony", "gala", "reception", "reunion", "summit", "competition", "tournament", "fair", "exhibition", "launch", "celebration", "party", "performance"]
    date_pattern = r'\b(?:\d{1,2}(?:st|nd|rd|th)?\s+\w+|\w+\s+\d{1,2}(?:st|nd|rd|th)?)\b'
    dates_found = re.findall(date_pattern, email_body)
    if dates_found:
        processed_date = preprocess_date_string(dates_found[0])
        event_date = parse(processed_date, fuzzy=True)
    for sentence in doc.sents:
        for keyword in event_keywords:
            if keyword in sentence.text.lower():
                event_title = summary(sentence.text)
                event_description = sentence.text
                break
    formatted_start_date = format_date_for_output(event_date)
    formatted_end_date = 'null'
    output = f'''
    {{
      event_id: {event_id},
      title: "{event_title}",
      start: {formatted_start_date},
      end: {formatted_end_date},
      description: "{event_description}",
      admin_id: {admin_ids},
    }}
    '''
    return output
# Example email body
for k in range(len(list2)):
  email_body=list2[k]
  output = extract_event_info(email_body)
  print(output)


    {
      event_id: 122,
      title: "Respected student, You are cordially invited",
      start: new Date("Tue Aug 15 2023 00:00:00 "),
      end: null,
      description: "Respected student,
You are cordially invited to our conference on 15-10-2023
",
      admin_id: [1, 2, 3, 4],
    }
    
